- Stitch together all tiles based on graph connections
- Calculate how many sequences are desired originals

In [1]:
import networkx as nx
import itertools
from difflib import SequenceMatcher
from Bio.Seq import Seq
import generate_tiles_basic as gt

tiles = gt.generate_tiles(gt.get_seqs("AllUniqueDNA.fasta"), gt.tile_length, gt.OVERLAP, gt.num_tiles)
THRESHOLD = 0.95

start = len(gt.start_overlap)
end = -len(gt.end_overlap)

seq_list = [seq for seq in tiles.values()]
tile_list = []
for seq in seq_list[:5]:
    for i, tile in enumerate(seq):
        if i == 0:
            tile_list.append([i, tile[start:]])
        elif i == len(seq)-1:
            tile_list.append([i, tile[:end]])
        else:
            tile_list.append([i, tile])
sources = []
targets = []
for tile in tile_list:
    if tile[0] == 0:
        sources.append(tile[1])
    elif tile[0] == 10:
        targets.append(tile[1])
tile_list

13 147
5' primer:  61.282551839883254
3' primer:  62.64545830315103
Unique 5' end fragments:  264
Unique 3' end fragments:  270


[[0,
  'ATGGAAAGAATAAAAGAACTACGGAATCTGATGTCGCAGTCTCGCACTCGCGAGATACTGACAAAAACCACAGTGGACCATATGGCCATAATTAAGAAGTACACATCGGGGAGACAGGAAAAGAACCCGTCACTTAGGATGAAATGGATGATGGCAATGAAATATCCAATCACTGCT'],
 [1,
  'ATGATGGCAATGAAATATCCAATCACTGCTGACAAAAGGGTAACAGAAATGGTTCCGGAGAGAAATGAACAAGGACAAACTCTATGGAGTAAAATGAGTGATGCTGGATCAGATAGAGTGATGGTATCACCTTTGGCTGTAACATGGTGGAATAGGAATGGACCCGTGACAAGTACGGTCCATTACCCAAAAGTGTACAAAACTTAT'],
 [2,
  'GTCCATTACCCAAAAGTGTACAAAACTTATTTCGACAAAGTCGAAAGGTTAAAACATGGAACCTTTGGCCCTGTCCATTTTAGAAATCAAGTCAAGATACGCAGAAGAGTAGACATAAACCCTGGTCATGCAGACCTCAGTGCCAAAGAGGCACAAGATGTAATTATGGAAGTTGTTTTTCCCAATGAAGTGGGAGCCAGAATACTA'],
 [3,
  'TTTCCCAATGAAGTGGGAGCCAGAATACTAACATCAGAATCACAACTAACAATAACTAAAGAGAAAAAAGAAGAACTCCGAGATTGCAAAATTTCTCCCTTGATGGTCGCATACATGTTAGAGAGAGAACTTGTGCGAAAAACAAGATTTCTCCCAGTTGCTGGCGGAACAAGCAGTATCTACATTGAAGTTTTACATTTGACTCAA'],
 [4,
  'ATCTACATTGAAGTTTTACATTTGACTCAAGGAACGTGTTGGGAACAAATGTACACTCCAGGTGGAGGAGTGAGGAATGACGATGTTGACCAAAGCCTAATTATTGCGGCCAGGAACATAGTAAGAAGAGCCGCAGTATCAGCAGA

In [2]:
def similar(overlap1, overlap2):
    return SequenceMatcher(None, overlap1, overlap2).ratio()

def gen_graph(tile_list, overlap, threshold, sources, targets):
    """
    tile_list: list of all tiles from all sequences
    overlap: length of overlaps
    threshold: fraction from 0 to 1 indicating percentage of paired bases needed for two tiles to anneal
    sources: list of 5' end fragments
    targets: list of 3' end fragments
    """
    T = nx.DiGraph()
    T.add_nodes_from(tile_list)
    
    for tile1, tile2 in itertools.combinations(tile_list, 2):
        if similar(tile1[-overlap:], tile2[:overlap]) > threshold:
            T.add_edge(tile1[:-overlap], tile2)
    print(T.edges())
    paths = []
    for source, target in itertools.product(sources, targets):
        for path in nx.all_simple_paths(T, source, target):
            print(path)
            #paths.append([path for path in nx.all_simple_paths(T, source, target)])
    return paths

paths = gen_graph([tile[1] for tile in tile_list], gt.OVERLAP, THRESHOLD, sources, targets)


[('...')]


In [ ]:
handle = open(in_file, 'w')
        SeqIO.write(records, handle, "fasta")
        out_file = "aligned.clustal"
        clustalomega_cline = cmd(infile=in_file, outfile=out_file, verbose=True, auto=True, outfmt="fasta", force=True, seqtype="DNA")
        clustalomega_cline()

In [ ]:
#dictionary of lengths of overlaps attached to each tile number